<a href="https://colab.research.google.com/github/CristianMorasso/statistical_project/blob/master/Copia_di_statiscal_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Learning Project: ALFA Dataset Anaysis


In [ ]:
# Download the dataset from our git repository hosted on Github
!pip install icecream sktime scikit-learn polars numpy
!rm -rf statiscal_project
!git clone https://github.com/CristianMorasso/statistical_project.git

fatal: destination path 'statistical_project' already exists and is not an empty directory.


In [ ]:
# Read the data from the csv files
import numpy as np
import pandas  as pd
import os
from random import shuffle

dataset_dir = "statistical_project/data"
dfs = [] # dataframe list

classToString= {0:"no_failure", 1:"elevator", 2:"engine", 3:"rudder",4:"ailerons"}

stringToClass = {}
for key, value in classToString.items():
  stringToClass[value] = key


for _, cl  in enumerate(os.listdir(dataset_dir)):
  path1 = os.path.join(dataset_dir, cl)

  # Get rid of files or bad dirs
  if not os.path.isdir(path1) or cl == "features":
    print(f"{cl} is not a dir or not useful!")
    continue

  for f in  os.listdir(path1):
    pathf = os.path.join(path1, f)
    df = pd.read_csv(pathf)

    # Remove Label field

    try:
      df["field.data"] = df["field.data"].apply(func=lambda x: stringToClass[cl] if int(x)>0 else 0)
      df.rename(columns={"field.data": "Y"})
    except:
      print(f"Error for {cl}")
      df["Y"]  = pd.Series([ 0 for _ in range(df.shape[0])])

    # Transforming to a pd.multi-indexed-DataFrame
    #
    # Add the colums for multi-indexing
    df.insert(0, "flight", [ i for _ in range(len(df))], True)
    df.insert(1, "step", [ a for a in range(len(df))], True)
    # And obv set the indexes for the df


    # Drop the time column!
    df = df.drop("%time", axis=1)

    # Save it
    dfs.append(df)

print(f"\nNumber of Time Series: {len(dfs)}")
assert len(dfs) == 45  # We know the exact number from the repo

# Shuffle the array of dataframes for so that classes are not contiguous
shuffle(dfs)

dfs[0].head()

Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
Error for no_failure
README.md is not a dir or not useful!
__init__.py is not a dir or not useful!
features is not a dir or not useful!

Number of Time Series: 45


,field.orientation.x,field.orientation.y,field.orientation.z,field.angular_velocity.x,field.angular_velocity.y,field.angular_velocity.z,field.linear_acceleration.x,field.linear_acceleration.y,field.linear_acceleration.z,field.data,...,field.alt_error,field.aspd_error,field.xtrack_error,delta_yaw,delta_pitch,delta_roll,delta_x,delta_y,delta_z,delta_roll_airspeed
0,-0.018369,-0.107484,-0.993491,0.779634,0.080698,-0.054738,1.608291,0.912018,11.434554,0,...,NaN,NaN,NaN,NaN,NaN,NaN,-13.328099,9.648238,1.324880,NaN
1,-0.018369,-0.107484,-0.993491,0.779634,0.080698,-0.054738,1.608291,0.912018,11.434554,0,...,-0.54,-382.258423,-35.322163,178.019249,-6.375798,27.482008,-1.363692,4.098843,1.326273,15.0
2,-0.018369,-0.107484,-0.993491,0.779634,0.080698,-0.054738,1.608291,0.912018,11.434554,0,...,-0.54,-382.258423,-35.322163,178.019249,-6.375798,27.482008,-1.363692,4.098843,1.326273,15.0
3,-0.018369,-0.107484,-0.993491,0.779634,0.080698,-0.054738,1.608291,0.912018,11.434554,0,...,-0.54,-382.258423,-35.322163,178.019249,-6.345798,27.482008,-1.492430,4.042004,1.380345,15.0
4,-0.018369,-0.107484,-0.993491,0.779634,0.080698,-0.054738,1.608291,0.912018,11.434554,0,...,-0.68,-382.258423,-78.401863,-47.980751,-8.685798,-57.397989,-1.660447,3.971760,1.408059,16.0


### Inpute the missing values and remove not usable features

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

# Apply the features Mask:
#
# finally use the mask on every timeseries of the OG dataframes list

# Remove all the features that are not shared by all Time Series
#
# Only use the variables that can be found in all the dataframes
features = []
# Get the biggest set:
# (We are considering that that the bigger set is just a superset and has all the other base features)
for df in dfs:
   features = max(features, df.columns.values, key=lambda x: len(x))
# Then get the set intersection of all of the features sets
features = set(features)
for df in dfs:
  features = features.intersection(set(df.columns.values))

# Apply the feature mask and inputer to all the datframe in our dataset
dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[list(features)]) , dfs))

<class 'numpy.ndarray'>
(1247, 26)


### Feature Selection

##### With SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest

# Concat all the time series in as single dataframe
concatted_dfs = pd.concat(dfs)
y = concatted_dfs["Y"]

# Print the number of nans per per each columns
print(concatted_dfs.isnull().sum())
print(f"Length of Concatted Dataframes: {len(concatted_dfs)}")

field.twist.linear.x           145
field.orientation.y            294
field.twist.angular.z          145
field.linear_acceleration.y    294
field.twist.linear.z           145
field.angular_velocity.y       294
field.aspd_error                57
field.xtrack_error              57
delta_x                         59
delta_z                         59
field.angular_velocity.z       294
field.twist.linear.y           145
field.alt_error                 57
field.orientation.z            294
delta_pitch                     57
field.linear_acceleration.x    294
delta_yaw                       57
delta_roll                      59
delta_y                         59
field.angular_velocity.x       294
field.orientation.x            294
field.twist.angular.y          145
delta_roll_airspeed             60
field.twist.angular.x          145
field.linear_acceleration.z    294
dtype: int64
Length of Concatted Dataframes: 96000


In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

# Inpute the nan values to the mean of the other entries value of the same column
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(concatted_dfs)
inp_out = imp.transform(concatted_dfs)

# Get the best 20 features of the dataset
columns_ = concatted_dfs.columns.values
feature_mask= SelectKBest(k=20).fit(inp_out, y).get_feature_names_out(columns_)

# ( Uses sklearn.feature_selection.f_classify )
print("Feature Mask:")
print(feature_mask)
print(f"Length of Feature Mask: {len(feature_mask)}")

In [ ]:
# APPLY THE FEATURES MASK:
#
# finally use the mask on every dataframe in the original dataframeList
# and also inpute but using time-series specific data:
dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x[feature_mask]) , dfs))

# Now we have the 20 best features without any nan value:
type(dfs[0]) # is a numpy.ndarray! wow!
print(dfs[0][:2,:]) # First 2 lines
print(dfs[0].shape) # shape of the array

[[-6.12688267e-02 -7.46916535e-02  7.84500438e-01 -4.09452055e-01
  -6.20859159e-02 -1.58507355e+02 -3.64743652e+01  2.95104289e+00
  -5.78435540e-01 -7.44200950e-02  1.69617445e-01  2.50000000e-01
  -4.96952845e-01  4.41096354e-02  1.72930016e+02  1.65214367e+01
  -6.64287492e-02 -6.19853966e-02  1.50000000e+01  1.05013892e+01]
 [-6.12688267e-02 -7.46916535e-02  7.84500438e-01 -4.09452055e-01
  -6.20859159e-02 -1.49973297e+02 -3.47822838e+01  1.88149457e+01
  -5.40839553e-01 -7.44200950e-02  1.69617445e-01  2.99999982e-01
  -4.96952845e-01  4.41096354e-02  1.72930016e+02  1.64879150e+01
  -6.64287492e-02 -6.19853966e-02  1.50000000e+01  1.05013892e+01]]
(2185, 20)


In [ ]:
# Now finally use the mask on every dataframe in the original
# dataframeList and also inpute but using time-series specific data:
new_dfs = list(map(lambda x: SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x) , new_dfs))

# Now we have the 20 best features without any nan value:
type(new_dfs[0]) # is a numpy.ndarray! wow!
print(new_dfs[0][:2,:]) # First 2 lines
print(new_dfs[0].shape) # shape of the array

[[-6.12688267e-02 -7.46916535e-02  7.84500438e-01 -4.09452055e-01
  -6.20859159e-02 -1.58507355e+02 -3.64743652e+01  2.95104289e+00
  -5.78435540e-01 -7.44200950e-02  1.69617445e-01  2.50000000e-01
  -4.96952845e-01  4.41096354e-02  1.72930016e+02  1.65214367e+01
  -6.64287492e-02 -6.19853966e-02  1.50000000e+01  1.05013892e+01]
 [-6.12688267e-02 -7.46916535e-02  7.84500438e-01 -4.09452055e-01
  -6.20859159e-02 -1.49973297e+02 -3.47822838e+01  1.88149457e+01
  -5.40839553e-01 -7.44200950e-02  1.69617445e-01  2.99999982e-01
  -4.96952845e-01  4.41096354e-02  1.72930016e+02  1.64879150e+01
  -6.64287492e-02 -6.19853966e-02  1.50000000e+01  1.05013892e+01]]
(2185, 20)


#### With randomForest scores:

In [ ]:
# WIP currently it doesn't work
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

def plot_feature_importances(model):
    fig_scale = 0.9
    n_features = X_train.shape[1]
    plt.figure(figsize=(7*fig_scale,5.4*fig_scale))
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), all_features)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

### Split the dataset into windows

In [ ]:
from sktime.split import SlidingWindowSplitter

splitted_dfs, window_length, step_length = [], 10, 7
splitter = SlidingWindowSplitter(window_length=window_length, step_length=step_length)
splitted_dfs = [ splitter.split(df)[0] for df in dfs] # We are ignoring the forecasting horizon
splitted_dfs = np.concatenate(splitted_dfs, axis=0 )

print(f"Length of windowed dataset: {len(splitted_dfs)}")

TypeError: 'generator' object is not subscriptable

In [ ]:
# Create a feature_mask with the n performing best features and then apply the feature_mask to all of our data

### Multi-Indexed Pandas Dataframe (sktime Panel)

In [ ]:
# Finally let's create a multiIndexed pandas.Dataframe so that we can use our data with the sktime library
_dfs = []
for i, npVals in enumerate(new_dfs):
  df = pd.DataFrame(npVals,columns=feature_mask)
  df.insert(0, "flight", [ i for _ in range(len(df))], True)
  df.insert(1, "step", [ a for a in range(len(df))], True)
  _dfs.append(df)
dfs = pd.concat(_dfs)

# Set the 2 Indexes (only flight is unique)
print(f"Before: {dfs.shape}")
dfs = dfs.drop_duplicates(subset=['flight', 'step'], keep='last')
dfs = dfs.set_index(["flight", "step"])
print(f"After: {dfs.shape}")

(96000, 22)
(96000, 20)


# Testing Models:

In [ ]:
# list all classifiers in sktime
#from sktime.registry import all_estimators
#all_estimators("classifier", as_dataframe=True) # , filter_tags={"capability:unequal_length":TrueNN

In [ ]:
# from sktime.datatypes import MTYPE_REGISTER
# pd.DataFrame(MTYPE_REGISTER)

In [ ]:
# from sktime.split import temporal_train_test_split
# from sktime.datatypes import check_raise
#
# from sklearn.pipeline import Pipeline
# from sktime.transformations.panel.padder import PaddingTransformer
# from sktime.transformations.panel.compose import ColumnConcatenator
# from sktime.transformations.panel.interpolate import TSInterpolator
#
# check_raise(dfs, mtype="pd-multiindex")
# check_raise(y_time_series, mtype='np.ndarray')
# dfs =  PaddingTransformer().fit_transform(dfs)
# y_train, y_test, X_train, X_test  = temporal_train_test_split(X=dfs,y=y_time_series, test_size=15)

In [ ]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.padder import PaddingTransformer
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.transformations.panel.interpolate import TSInterpolator

cnn = CNNClassifier(n_epochs=2000, batch_size=4, kernel_size=3, avg_pool_size=3, n_conv_layers=3, verbose=True, loss='binary_crossentropy')
cnn.fit(X_train, y_train)
cnn.score(X_test,y_test)

#clf = Pipeline([
#    ("transform", TSInterpolator(50)),
#     ("concatenate", ColumnConcatenator()),
#     ("classify", TimeSeriesForestClassifier(n_estimators=100)),
#    ("transform", PaddingTransformer()),
#    ("classify", RandomIntervalClassifier(n_intervals=10)),
#])
#clf.fit(X_train, y_train)
#clf.score(X_test,y_test)

/usr/local/lib/python3.10/dist-packages/sktime/base/_base_panel.py:542: UserWarning: only single label seen in y passed to fit of classifier CNNClassifier
  warn(


0.3333333333333333

In [ ]:
from sktime.classification.deep_learning.rnn import SimpleRNNClassifier
from sklearn.model_selection import LearningCurveDisplay

rnn = SimpleRNNClassifier(n_epochs=20, batch_size=20)
rnn.fit(X_train, y_train)
rnn.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sktime/base/_base_panel.py:542: UserWarning: only single label seen in y passed to fit of classifier SimpleRNNClassifier
  warn(


0.3333333333333333